# Bitcoin Price Prediction :
1. Bitcoin (₿) is a cryptocurrency, a form of electronic cash. It is a decentralized digital currency without a central bank or single administrator, though some researchers point at a trend towards centralization.

2. Bitcoins can be sent from user to user on the peer-to-peer bitcoin network directly, without the need for intermediaries, though intermediaries are widely used.Transactions are verified by network nodes through cryptography and recorded in a public distributed ledger called a blockchain.

<b> Dataset Obtained using the API -> https://min-api.cryptocompare.com/ </b>

<b> Dataset Properties </b>
1. Number of Rows -> 2001
2. Number of Columns/Features -> 7
3. Historical Data obtained using the API above.

<b> Columns/Features </b>
1. Close (Target Variable/Feature) - Bitcoin price in USD at the time of closing.
2. Open - Bitcoin Price at the time of opening.
3. High - Highest Price in USD of a bitcoin during a transanction period.
4. Low - Lowest Price in USD of a bitcoin during a transanction period.
5. Time - Date/Time of the transanction period. (Here it denotes just the day transanction opened)
6. VolumeFrom - Total amount of bitcoin exchange that took place during the transanction period.
7. VolumeTo - Total amount of USD exchange that took place during the transanction period.

Note -> Here transanction period is a day

## Problem statement :
<b> Given the historical data about the bitcoin exhanges, predict the close price of bitcoin the next(future) days </b>

## Constraints :
1. Slightly lesser Latency (as the prices of bitcoin changes rapidly we need to calculate the predicted price in somewhat reasonable time).
2. Interpretability is not important.
3. Errors are costly. ( Costly, but not much, a person wont die if he loses some money :) )

## Importing Libraries :

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import datetime
%matplotlib inline

## Getting the dataset using the API :

In [47]:
json_data = requests.post("https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&limit=2000").json()

In [48]:
data = pd.DataFrame(json_data['Data'])
data.tail()

,close,high,low,open,time,volumefrom,volumeto
1996,7610.90,7760.74,7449.31,7736.25,1533081600,78466.37,5.954028e+08
1997,7542.34,7713.32,7470.86,7610.90,1533168000,55711.79,4.240936e+08
1998,7417.60,7542.49,7296.76,7542.34,1533254400,79126.57,5.869825e+08
1999,7017.89,7490.89,6940.15,7417.60,1533340800,74300.44,5.340029e+08
2000,6985.16,7092.53,6898.31,7017.89,1533427200,26535.17,1.860206e+08


In [49]:
data.shape

(2001, 7)

## Data Preprocessing :

In [50]:
data.isnull().sum()

close         0
high          0
low           0
open          0
time          0
volumefrom    0
volumeto      0
dtype: int64

In [52]:
data.describe()

,close,high,low,open,time,volumefrom,volumeto
count,2001.000000,2001.000000,2001.000000,2001.00000,2.001000e+03,2001.000000,2.001000e+03
mean,1964.493298,2032.874563,1878.023093,1961.01927,1.447027e+09,63789.533158,2.056642e+08
std,3316.758668,3452.568702,3140.223879,3315.16158,4.992047e+07,59956.985027,4.931185e+08
min,24.200000,25.830000,21.720000,24.20000,1.360627e+09,404.380000,2.101127e+05
25%,265.460000,273.570000,259.000000,265.03000,1.403827e+09,22803.350000,6.837219e+06
50%,518.090000,540.880000,499.710000,517.94000,1.447027e+09,48878.180000,1.893281e+07
75%,1179.220000,1211.740000,1131.350000,1177.33000,1.490227e+09,86516.580000,9.245835e+07
max,19345.490000,19870.620000,18750.910000,19346.60000,1.533427e+09,572349.320000,6.245732e+09


In [53]:
data['time'] = data['time'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime("%Y-%m-%d"))

In [54]:
data.tail()

,close,high,low,open,time,volumefrom,volumeto
1996,7610.90,7760.74,7449.31,7736.25,2018-08-01,78466.37,5.954028e+08
1997,7542.34,7713.32,7470.86,7610.90,2018-08-02,55711.79,4.240936e+08
1998,7417.60,7542.49,7296.76,7542.34,2018-08-03,79126.57,5.869825e+08
1999,7017.89,7490.89,6940.15,7417.60,2018-08-04,74300.44,5.340029e+08
2000,6985.16,7092.53,6898.31,7017.89,2018-08-05,26535.17,1.860206e+08


## Exploratory Data Analysis :